In [1]:
# def load_data(loc:str, index:list):
#     df = pandas.read_csv(f'data/{loc}_solar19.csv')
#     df['idx'] = index
#     df = df.set_index('idx')
#     return df
# idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]
# weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
# weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
# weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
# weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]""
# demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
# demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
# demand_ny = DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga
# ng_price = make_henry_price_df(
#     file_name='data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
# ng_price = ng_price.drop(columns= 'scales')
# ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
# ng_price = pandas.concat([ng_price])

In [2]:
import sys

sys.path.append('src')

In [3]:
from energiapy.blocks.model import Model
from energiapy.blocks.taskmaster import TaskMaster
from dataclasses import dataclass, field, fields, asdict
from operator import is_

In [4]:
from pandas import DataFrame
from energiapy.components import *
from energiapy.model.scenario import Scenario
from energiapy.parameters.designators.mode import X
from energiapy.parameters.designators.incidental import I

In [5]:
from energiapy.core.aliases.is_input import IsBoundInput

In [6]:
from energiapy.components._base._scope import _Scope
from energiapy.components._base._defined import _Defined
from energiapy.components.temporal.scale import Scale
from energiapy.components.spatial.location import Location
from energiapy.components.spatial._spatial import _Spatial
from energiapy.core._handy._enums import _Dummy
from energiapy.indices.disposition import Disposition

In [7]:
s = Scenario(default=True)

In [8]:
# s.h2 = Horizon({'days': 2, 'hours': 12})
s.hor = Horizon(discretizations=[2, 12])
s.scales
s.net = Network(['madgaon', 'ponje', 'cacoda'])  # , link_all=True)
s.nodes
s.alink = Linkage(source=s.madgaon, sink=s.ponje, bi=True, distance=50)
s.blink = Linkage(source=s.ponje, sink=s.cacoda, bi=False, distance=80)
s.clink = Linkage(source=s.cacoda, sink=s.madgaon, bi=True, distance=100)
s.dlink = Linkage(source=s.madgaon, sink=s.ponje, bi=False, distance=200)

In [9]:
a = DataFrame({'a': list(range(2))})
b = DataFrame({'b': list(range(24))})

In [10]:
s.csh = Cash(
    spend={
        s.madgaon: True,
        s.ponje: {s.t2: (2, a)},
        s.cacoda: a,
        s.network: {s.t2: [5, 7]},
    },
    # label='cash',
    basis='USD',
)

c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:227: InconsistencyWarning: 
csh.spend:Inconsistent temporal scale for ponje at t2.
Updating to t1
Set Scneario.ok_inconsistent=False, to enforce strictly
  warn(


In [11]:
s.cash.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [12]:
s.H2 = Resource(
    # sell={s.t0: [0, 4], s.t1: [a, 34], s.t2: (a, b)},
    price_sell={
        s.madgaon: {s.t1: (2, a)},
        s.ponje: {s.t1: (2, b)},
        s.cacoda: 300,
    },
)

c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:227: InconsistencyWarning: 
H2.price_sell:Inconsistent temporal scale for ponje at t1.
Updating to t2
Set Scneario.ok_inconsistent=False, to enforce strictly
  warn(


In [13]:
s.H2.parameters

[Price^sell[csh, H2, cacoda, t0],
 Price^sell[csh, H2, madgaon, t1],
 Price^sell[csh, H2, ponje, t2]]

In [14]:
s.H2.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [15]:
s.Solar = Resource(buy=True, basis='MW', label='Solar Power')


s.Wind = Resource(buy=a, basis='MW', label='Wind Power')


s.Power = Resource(basis='MW', label='Power generated')


s.Uranium = Resource(
    buy=DataFrame({'a': [i for i in range(24)]}),
    price_buy=42.70 / (250 / 2),
    basis='kg',
    label='Uranium',
)


# s.H2_L = Resource(sell=(0, 23), basis='tons', label='Hydrogen')


# s.CO2_AQoff = Resource(basis='tons', label='Carbon dioxide - sequestered')


s.H2O = Resource(buy=(20, 50), price_buy=b, basis='tons', label='Water')


s.CH4 = Resource(buy=[20, 40], price_buy=20, basis='tons', label='Natural gas')


s.CO2 = Resource(basis='tons', label='Carbon dioxide', block='Resource')


s.CO2_Vent = Resource(
    basis='tons',
    label='Carbon dioxide - Vented',
    price_sell=(2, 20),
    emission_sell={s.gwp: b},
)


s.O2 = Resource(sell=[20, True], basis='tons', label='Oxygen')


s.CO2_DAC = Resource(basis='tons', label='Carbon dioxide - captured')
s.Power = Resource(buy=[0, a], basis='MW', label='Power generated')

c:\Users\cacodcar\energiapy\src\energiapy\model\_ok.py:54: OverWriteWarning: 
A Power was already defined.
Overwriting. Set Scenario.ok_overwrite=False, to enforce strictly
This should not cause any modeling issues.
Check Scenario defaults if unintended.
Only one instance of Horizon, Network, Cash, and Land are allowed
  warn(OverWriteWarning(cmp))


In [16]:
s.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [17]:
s.LiR = Material(
    emission_use={s.gwp: {s.t2: 1.484}},
    use={s.ponje: 1.5},
    basis='kg',
    label='Lithium Reserves',
    citation='Nelson Bunyui Manjong (2021)',
)

In [18]:
s.LiR.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [19]:
s.WF = Process(
    conversion={s.Power: {X(0): {s.Wind: -1}, X(1): {s.Wind: -1}}},
    produce=[(0, 3), (3, 5)],
    capex={s.madgaon: {s.t1: {1462000, I(202233)}}},
    opex={4953, I(70)},
    capacity=1000,
    emission_setup={s.gwp: 50, s.odp: 2900},
    use_material={s.LiR: 1.5},
    locations=[s.madgaon, s.ponje],
    use_land=30,
    label='Wind mill array',
    basis='MW',
)


s.PV = Process(
    price_sell={s.Power: 30},
    credit={s.Power: DataFrame({'a': [2]})},
    conversion={s.Power: {X(1): {s.Solar: -1.2}, X(2): {s.Solar: -1.5}}},
    capex={X(0): 1333, X(1): 1444, X(2): 1555},
    opex=22623,
    capacity={X(0): 3000, X(1): 4000, X(2): 5000},
    locations=s.cacoda,
    label='Solar PV',
    basis='MW',
)


s.SMRH = Process(
    use_land=50,
    conversion={
        s.H2: {
            s.Power: -1.11,
            s.CH4: -3.76,
            s.H2O: -23.7,
            s.CO2_Vent: 1.03,
            s.CO2: 9.332,
        }
    },
    capex={
        2520000,
    },
    opex={51.5, I(945000)},
    capacity=[1000],
    label='Steam methane reforming + CCUS',
)

s.NGCC = Process(
    price_buy={s.CH4: 4, s.H2O: 20},
    conversion={
        s.Power: {
            s.CH4: -0.108,
            s.H2O: -10,
            s.CO2_Vent: 0.297 * 0.05,
            s.CO2: 0.297 * 0.95,
        }
    },
    capex=2158928,
    opex={4090, I(53320)},
    capacity=[1, 100],
    label='NGCC + 95% CC',
)
s.SMR = Process(
    capex=2400,
    opex={0.03, I(800)},
    conversion={s.H2: {s.Power: -1.11, s.CH4: -3.76, s.H2O: -23.7, s.CO2_Vent: 9.4979}},
    capacity=1000,
    label='Steam methane reforming',
)
s.H2FC = Process(
    price_buy={s.H2: 2},
    sell={s.Power: 20},
    conversion={s.Power: {s.H2: -0.050}},
    capex=1.6 * 10**6,
    opex=3.5,
    capacity=[1000, 2000],
    label='hydrogen fuel cell',
)
s.DAC = Process(
    capex=0.02536,
    opex=0.634,
    conversion={s.CO2_DAC: {s.Power: -0.193, s.H2O: -4.048}},
    capacity=1000,
    label='Direct air capture',
)
# s.PSH = Process(conversion = {s.Power: 0.6}, capex = 3924781, fopex= 17820, vopex = 512.5, store = 10000, capacity=1000, label='Pumped storage hydropower', basis = 'MW')
s.ASMR = Process(
    buy={s.Uranium: 40},
    conversion={s.Power: {s.Uranium: -4.17 * 10 ** (-5), s.H2O: -3.364}},
    capex=7988951,
    opex=I(0.04 * 0.730),
    capacity=1000,
    label='Small modular reactors (SMRs)',
    locations=s.madgaon,
)
s.AWE = Process(
    use_land=(0, 20),
    conversion={s.H2: {s.Power: -1, s.O2: 0.7632, s.H2O: -0.1753}},
    capex={1.1 * 10**6, I(20)},
    opex=I(16918),
    capacity=True,
    label='Alkaline water electrolysis (AWE)',
    citation='Demirhan et al. 2018 AIChE paper',
)

In [20]:
s.LiI = Storage(capacity=[2000], inventory=s.Power, capacity_in=[40], capacity_out=[20])
s.LiI2 = Storage(capacity=[2000], inventory={s.Power: 0.6})
s.H2Stg = Storage(
    capacity=[2000], inventory={s.H2: {X(0): {s.Power: 0.8}, X(1): {s.Power: 0.9}}}
)

In [36]:
print(len(s.variables))
print(len(set(s.variables)))
print(len(set([v.name for v in s.variables])))
print()
print(len(s.parameters))
print(len(set(s.parameters)))
print(len(set([v.name for v in s.parameters])))
print()
print(len(s.constraints))
print(len(set(s.constraints)))
print(len(set([v.name for v in s.constraints])))
print()
print(len(s.dispositions))
print(len(set(s.dispositions)))
print(len(set([v.name for v in s.dispositions])))

232
232
232

167
167
167

167
124
124

100
61
61


167
167


167
124


100
61


In [34]:
for block in s.program.blocks:
    print(block.attr_dispositions)

{'use': [], 'cost_use': [], 'emission_use': []}
{'emit': []}
{'emit': []}
{'emit': []}
{'emit': []}
{'emit': []}
{'emit': []}
{'emit': []}
{'emit': []}
{'emit': []}
{'emit': []}
{'emit': []}
{'has': [], 'needs': []}
{'has': [], 'needs': []}
{'has': [], 'needs': []}
{'has': [], 'needs': []}
{'spend': [], 'earn': []}
{'ship': [], 'buy': [], 'sell': [], 'price_buy': [], 'price_sell': [(H2, cacoda, t0), (H2, madgaon, t1), (H2, ponje, t2)], 'credit': [], 'penalty': [], 'emission_buy': [], 'emission_sell': [], 'emission_loss': []}
{'ship': [], 'buy': [], 'sell': [], 'price_buy': [], 'price_sell': [], 'credit': [], 'penalty': [], 'emission_buy': [], 'emission_sell': [], 'emission_loss': []}
{'ship': [], 'buy': [], 'sell': [], 'price_buy': [], 'price_sell': [], 'credit': [], 'penalty': [], 'emission_buy': [], 'emission_sell': [], 'emission_loss': []}
{'ship': [], 'buy': [], 'sell': [], 'price_buy': [], 'price_sell': [], 'credit': [], 'penalty': [], 'emission_buy': [], 'emission_sell': [], 'emi

In [21]:
is_(s.program.variables[5], s.program.variables[6])

False

In [25]:
len(set(s.program.variables))

232

In [22]:
s.program.variables[6]

buy[CH4, net, t0]

In [25]:
s.LiI.capacity

capacity{(LiI, net, t0): [0[LiI, net, t0], 2000[LiI, net, t0]]}

In [24]:
getattr(s.LiI, 'capacity')

capacity{(LiI, net, t0): [0[LiI, net, t0], 2000[LiI, net, t0]]}

In [35]:
s.attr.bounds()

['has',
 'needs',
 'spend',
 'earn',
 'emit',
 'use',
 'ship',
 'buy',
 'sell',
 'capacity',
 'produce',
 'store',
 'transport']

In [37]:
from energiapy.core.aliases.is_input import IsBoundInput

In [38]:
IsBoundInput

typing.Union[float, int, pandas.core.frame.DataFrame, typing.Tuple[typing.Union[float, int, pandas.core.frame.DataFrame], typing.Union[float, int, pandas.core.frame.DataFrame]], energiapy.parameters.values.theta.Theta, typing.Literal[True], energiapy.parameters.values.m.M, typing.List[typing.Union[float, int, pandas.core.frame.DataFrame, typing.Tuple[typing.Union[float, int, pandas.core.frame.DataFrame], typing.Union[float, int, pandas.core.frame.DataFrame]], energiapy.parameters.values.theta.Theta, typing.Literal[True], energiapy.parameters.values.m.M]], typing.Dict[energiapy.components.temporal.scale.Scale, typing.Union[float, int, pandas.core.frame.DataFrame, typing.Tuple[typing.Union[float, int, pandas.core.frame.DataFrame], typing.Union[float, int, pandas.core.frame.DataFrame]], energiapy.parameters.values.theta.Theta, typing.Literal[True], energiapy.parameters.values.m.M, typing.List[typing.Union[float, int, pandas.core.frame.DataFrame, typing.Tuple[typing.Union[float, int, panda

In [36]:
s.attr.exacts()

['capex',
 'opex',
 'cost_use',
 'price_buy',
 'price_sell',
 'credit',
 'penalty',
 'emission_setup',
 'emission_use',
 'emission_buy',
 'emission_sell',
 'emission_loss',
 'emission_use_land',
 'emission_use_material',
 'cost_use_land',
 'cost_use_material',
 'use_land',
 'use_material',
 'loss_transit',
 'loss_storage']

In [30]:
s.components

{'lnd_def': lnd_def,
 'gwp': gwp,
 'ap': ap,
 'epm': epm,
 'epf': epf,
 'ept': ept,
 'pocp': pocp,
 'odp': odp,
 'adpmn': adpmn,
 'adpmt': adpmt,
 'adpf': adpf,
 'wdp': wdp,
 'dm': dm,
 'market': market,
 'consumer': consumer,
 'earth': earth,
 'hor': hor,
 't0': t0,
 't1': t1,
 't2': t2,
 'net': net,
 'madgaon': madgaon,
 'ponje': ponje,
 'cacoda': cacoda,
 'alink': alink,
 'alink_': alink_,
 'blink': blink,
 'clink': clink,
 'clink_': clink_,
 'dlink': dlink,
 'csh': csh,
 'H2': H2,
 'Solar': Solar,
 'Wind': Wind,
 'Power': Power,
 'Uranium': Uranium,
 'H2O': H2O,
 'CH4': CH4,
 'CO2': CO2,
 'CO2_Vent': CO2_Vent,
 'O2': O2,
 'CO2_DAC': CO2_DAC,
 'LiR': LiR,
 'WF': WF,
 'PV': PV,
 'SMRH': SMRH,
 'NGCC': NGCC,
 'SMR': SMR,
 'H2FC': H2FC,
 'DAC': DAC,
 'ASMR': ASMR,
 'AWE': AWE,
 'LiI': LiI,
 'LiI_Power': LiI_Power,
 'LiI_in': LiI_in,
 'LiI_out': LiI_out,
 'LiI2': LiI2,
 'LiI2_Power': LiI2_Power,
 'LiI2_in': LiI2_in,
 'LiI2_out': LiI2_out,
 'H2Stg': H2Stg,
 'H2Stg_H2': H2Stg_H2,
 'H2Stg_

In [23]:
s.train = Transit(
    freight=s.H2,
    ship={s.H2: [300]},
    capacity=[5, 400],
    use_land=20,
    emission_setup={s.odp: 0.1},
    capex={s.alink: 200, s.blink: 300, s.clink: 400},
    opex=100,
    label='Train',
    basis='passengers',
    linkages=[s.alink],
)

making new at capacity[train, net, t0] (train, net, t0) train
making new at capacity[train, net, t0] (train, net, t0) train


CacodcarError: use^setup[lnd_def, train, cacoda, t0]:['lnd', 'trn', 'loc', 'scl'] not in [['lnd', 'trn', 'lnk', 'scl'], ['lnd', 'trn', 'lnk', 'scl', 'mde'], ['lnd', 'trn', 'ntw', 'scl'], ['lnd', 'trn', 'ntw', 'scl', 'mde'], ['lnd', 'lnk', 'scl'], ['lnd', 'lnk', 'scl', 'mde'], ['lnd', 'ntw', 'scl'], ['lnd', 'ntw', 'scl', 'mde'], ['mat', 'trn', 'lnk', 'scl'], ['mat', 'trn', 'lnk', 'scl', 'mde'], ['mat', 'trn', 'ntw', 'scl'], ['mat', 'trn', 'ntw', 'scl', 'mde'], ['mat', 'lnk', 'scl'], ['mat', 'lnk', 'scl', 'mde'], ['mat', 'ntw', 'scl'], ['mat', 'ntw', 'scl', 'mde'], ['ply', 'lnd', 'trn', 'lnk', 'scl'], ['ply', 'lnd', 'trn', 'lnk', 'scl', 'mde'], ['ply', 'lnd', 'trn', 'ntw', 'scl'], ['ply', 'lnd', 'trn', 'ntw', 'scl', 'mde'], ['ply', 'lnd', 'lnk', 'scl'], ['ply', 'lnd', 'lnk', 'scl', 'mde'], ['ply', 'lnd', 'ntw', 'scl'], ['ply', 'lnd', 'ntw', 'scl', 'mde'], ['ply', 'mat', 'trn', 'lnk', 'scl'], ['ply', 'mat', 'trn', 'lnk', 'scl', 'mde'], ['ply', 'mat', 'trn', 'ntw', 'scl'], ['ply', 'mat', '

In [ ]:
s.pprint()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>